## SCAD, pytorch, variable selection

In [1]:
!pip install torch

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

In [27]:
device = torch.device('cpu')
dtype = torch.float64

In [28]:
def scad_penalty(beta_hat, lambda_val, a_val):
    abs_beta_hat = torch.abs(beta_hat)
    is_linear = (abs_beta_hat <= lambda_val) # defining the flat outer bounds
    is_quadratic = torch.logical_and(lambda_val < abs_beta_hat, abs_beta_hat <= a_val * lambda_val) # if lambda < absolute value of beta <= a * lambda
    is_constant = (a_val * lambda_val) < abs_beta_hat # if absolute value of beta is greater than a * lambda
    
    linear_part = lambda_val * abs_beta_hat * is_linear
    quadratic_part = (2* a_val * lambda_val * abs_beta_hat - beta_hat**2 - lambda_val**2) / (2*(a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2*(a_val+1)) / 2 * is_constant

    return linear_part + quadratic_part + constant_part

def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val*((beta_hat <= lambda_val) + (a_val*lambda_val-beta_hat)*((a_val*lambda_val-beta_hat) > 0)/((a_val-1)*lambda_val)*(beta_hat>lambda_val))

In [98]:
scad_derivative(torch.tensor([.5,.5], dtype= dtype, device= device), 1, 1)

tensor([nan, nan], dtype=torch.float64)

Create your own PyTorch class that implements the method of SCAD regularization and variable selection (smoothly clipped absolute deviations) for linear models. Your development should be based on the following references:
https://andrewcharlesjones.github.io/journal/scad.html
https://www.jstor.org/stable/27640214?seq=1

Test your method on a real data set and determine a variable selection based on features' importance, according to SCAD.

In [102]:
class SCADLinear(nn.Module):
    def __init__(self, input_size, lambda_val, a_val):
        super(SCADLinear, self).__init__() # init nn.Module
        self.input_size = input_size
        self.lambda_val = lambda_val
        self.a_val = a_val

        self.linear = nn.Linear(input_size, 1, bias= False, device= device, dtype= dtype)

    def forward(self, x): # how necessary is this?
        # train the linear model on x
        return self.linear(x)
    
    def scad_derivative(self, beta_hat):
        solution = self.lambda_val*((beta_hat <= self.lambda_val) + (self.a_val*self.lambda_val-beta_hat)*((self.a_val*self.lambda_val-beta_hat) > 0)/((self.a_val-1)*self.lambda_val)*(beta_hat>self.lambda_val))
        return solution
    
    def loss_with_scad(self, y_pred, y_true):
        mse = nn.MSELoss()(y_pred,y_true)
        penalty = torch.squeeze(self.scad_derivative(beta_hat=self.linear.weight))[1]
        return mse + penalty
    
    def fit(self, x, y, num_epochs= 200, learning_rate= .001):
        optimize = optim.SGD(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimize.zero_grad() # clear the optimizer
            y_pred = self(x)
            loss_with_scad = self.loss_with_scad(y_pred, y)
            loss_with_scad.backward() # computes gradient
            optimize.step()

            if (epoch+1) % 100 == 0:
                print(f'epoch: {epoch+1}/{num_epochs}, loss_with_scad: {loss_with_scad.item()}')
        return
    
    def predict(self, x):
        self.eval()
        with torch.no_grad(): # forces custom gradient
            y_pred = self(x)
        return y_pred
    
    def get_coefficients(self):
        return self.linear.weight

In [4]:
import numpy as np
from scipy.linalg import toeplitz

In [22]:
# we want to define a function for generating x with a prescribed number of obsvervations, features and Toeplitz correlation structure.
def make_correlated_features(num_samples,p,rho):
  vcor = [] 
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x

In [41]:
rho = 0.9
p = 20
n = 150
x = make_correlated_features(n, p, rho)

In [42]:
beta =np.array([-1,2,3,0,0,0,0,2,-1,4])
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)
y = x@betastar + 1.5*np.random.normal(size=(n,1))

In [82]:
x_tensor = torch.tensor(x, device=device)
y_tensor = torch.tensor(y, device= device)

In [104]:
model = SCADLinear(x_tensor.shape[1], 0.5, 3)

In [105]:
model.fit(x_tensor,y_tensor, num_epochs=1000, learning_rate=0.01)

epoch: 100/1000, loss_with_scad: 3.5206135602450828
epoch: 200/1000, loss_with_scad: 2.9271076025363185
epoch: 300/1000, loss_with_scad: 2.5865521456803466
epoch: 400/1000, loss_with_scad: 2.3638036336341264
epoch: 500/1000, loss_with_scad: 2.2090046517173842
epoch: 600/1000, loss_with_scad: 2.0970431059949575
epoch: 700/1000, loss_with_scad: 2.013741314192788
epoch: 800/1000, loss_with_scad: 1.9504661119917728
epoch: 900/1000, loss_with_scad: 1.901651613498697
epoch: 1000/1000, loss_with_scad: 1.863542907286052


In [110]:
from sklearn.metrics import mean_squared_error as mse

In [111]:
mse(model.predict(x_tensor).detach().numpy(), y_tensor.detach().numpy())

1.86320628220977

Based on the simulation design explained in class, generate 200 data sets where the input features have a strong correlation structure (you may consider a 0.9) and apply ElasticNet, SqrtLasso and SCAD to check which method produces the best approximation of an ideal solution, such as a "betastar" you design with a sparsity pattern of your choice.

Use the methods you implemented above to determine a variable selection for the Concrete data set with quadratic interaction terms (polynomial features of degree 2). To solve this, you should consider choosing the best weight for the penalty function. What is the ideal model size (number of variables with non-zero weights), and what is the cross-validated mean square error?